In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Load the dataset
data = pd.read_csv("../dataset/emotions.csv")

# Data preprocessing
X = data['text']
y = data['label']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding sequences
max_length = 100  # Define your maximum sequence length
X = pad_sequences(X, maxlen=max_length)

# Convert labels to one-hot encoding
num_classes = len(data['label'].unique())
y = pd.get_dummies(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the RNN model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Predict labels
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.array(y_test), axis=1)

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print('Precision:', precision)
print('Recall:', recall)
print('F1-score:', f1)


Epoch 1/10
8337/8337 [==============================] - 728s 87ms/step - loss: 0.2017 - accuracy: 0.9073 - val_loss: 0.1206 - val_accuracy: 0.9294
Epoch 2/10
8337/8337 [==============================] - 726s 87ms/step - loss: 0.0977 - accuracy: 0.9389 - val_loss: 0.0965 - val_accuracy: 0.9368
Epoch 3/10
8337/8337 [==============================] - 737s 88ms/step - loss: 0.0904 - accuracy: 0.9421 - val_loss: 0.0964 - val_accuracy: 0.9365
Epoch 4/10
8337/8337 [==============================] - 736s 88ms/step - loss: 0.0859 - accuracy: 0.9435 - val_loss: 0.0970 - val_accuracy: 0.9317
Epoch 5/10
8337/8337 [==============================] - 743s 89ms/step - loss: 0.0827 - accuracy: 0.9459 - val_loss: 0.1018 - val_accuracy: 0.9329
Epoch 6/10
2606/2606 [==============================] - 50s 19ms/step - loss: 0.0960 - accuracy: 0.9363
Test Loss: 0.0960470661520958
Test Accuracy: 0.9363498687744141
2606/2606 [==============================] - 57s 21ms/step
Precision: 0.9227389057706633
Recall: 

In [6]:
# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')


print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Test Loss: 0.0960470661520958
Test Accuracy: 0.9363498687744141
Precision: 0.9373856932668088
Recall: 0.9363498956359012
F1 Score: 0.9346022692246436


In [9]:
model.save('emotions_rnn_new.keras')

In [6]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
data = pd.read_csv("../dataset/emotions.csv")

# Data preprocessing
X = data['text']
y = data['label']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Padding sequences
max_length = 100  # Define your maximum sequence length
X = pad_sequences(X, maxlen=max_length)

# Convert labels to one-hot encoding
num_classes = len(data['label'].unique())
y = pd.get_dummies(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


saved_model = load_model('emotions_rnn_new.keras')

loss, accuracy = saved_model.evaluate(X_test, y_test)

# 4. Make predictions using the model
y_pred = saved_model.predict_classes(X_test)

# 5. Calculate additional metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

/home/mizhap/.local/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 26 variables whereas the saved optimizer has 20 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 1 objects could not be loaded. Example error message for object <LSTMCell name=lstm_cell, built=True>:

Layer 'lstm_cell' expected 3 variables, but received 0 variables during loading. Expected: ['kernel', 'recurrent_kernel', 'bias']

List of objects that could not be loaded:
[<LSTMCell name=lstm_cell, built=True>]

In [1]:
from keras.models import load_model
from sklearn.metrics import precision_score, recall_score, f1_score


saved_model = load_model('emotions_rnn_new.keras')

loss, accuracy = saved_model.evaluate(X_test, y_test)

# 4. Make predictions using the model
y_pred = saved_model.predict_classes(X_test)

# 5. Calculate additional metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)


print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

2024-05-01 12:42:51.385046: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-01 12:42:51.390014: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-01 12:42:51.451667: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 12:42:51.451717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 12:42:51.453519: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 100], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']